# LLM4LLU: Speech To Text (STT) and Text To Speech (TTS) for Urdu

In our project, we aim to support both text and audio input/output modalities. Users should be able to send the query to the LLM through WhatsApp using a voice message if they so desire. Furthermore, users should also have the option of converting the LLM response into audio format as well if they wish to listen to the output. There are plently of well-documented STT and TTS models for English with impressive performance, therefore this notebook will only focus on using models which work well with Urdu and evaluating their performance. Supporting Urdu is critical for our application since we expect most users to converse with the LLM in Urdu, given the fact that they will be from low-literacy backgrounds and hence may not have the required level of fluency in English due to educational inequities that exist in Pakistan.

## STT with whisper:

For converting Urdu speech into text, we decided to use the `whisper` model by OpenAI (https://huggingface.co/openai/whisper-large-v3). `whisper` is a general-purpose speech recognition model, trained on a large dataset of diverse audio with impressive results in a variety of languages. With it's native support for Urdu, we will use `whisper` to transcribe three audio recordings in Urdu into text.

Note that `whisper` is also able to translate audio from a source langauge to English as well. However, anecdotally speaking, we found the translation results to be poor and not sensitive enough to capture the nuances present in the input recording. Therefore we will only use `whisper` for transcribing audio into the Urdu (Arabic) script, and passing that as a prompt to GPT4 directly. Given `GPT4`'s impressive capabilities in understanding and responding in a variety of languages, passing Urdu prompts should not pose to be a problem.

In [3]:
# !pip install --upgrade pip
# !pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate

In [1]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import IPython

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [6]:
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bia

In [7]:
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
sample1 = 'urdu_audio_1.ogg'
sample2 = 'urdu_audio_2.ogg'
sample3 = 'urdu_audio_3.ogg'
sound1 = IPython.core.display.display(IPython.display.Audio(sample1))
sound2 = IPython.core.display.display(IPython.display.Audio(sample2))
sound3 = IPython.core.display.display(IPython.display.Audio(sample3))
result1 = pipe(sample1, generate_kwargs={"language": "urdu"})
result2 = pipe(sample2, generate_kwargs={"language": "urdu"})
result3 = pipe(sample3, generate_kwargs={"language": "urdu"})
print("Sample 1:", result1["text"])
print("Sample 2:", result2["text"])
print("Sample 3:", result3["text"])

Sample 1:  یہ نیا پروجیکٹ کب تک مکمل ہو جائے گا؟
Sample 2:  مجھے یہ فارم کس تاری کو جمع کرانا ہے؟
Sample 3:  مجھے عید کے لئے کس رنگ کا کرتہ پہننا چاہیے؟


As we can see, despite some very minor spelling errors due to variances in how words may be enunciated by different people, whisper performs well and is able to output readable Urdu sentences by capturing what is being said in the audio recordings.

# TTS with Massively Multilingual Speech (MMS):

For converting Urdu responses into Urdu audio, we decide to use Facebook's `mms-tts` model, specifically the `urd-script_arabic` version (https://huggingface.co/facebook/mms-tts-urd-script_arabic). This model is part of Facebook's Massively Multilingual Speech project, aiming to provide speech technology across a diverse range of languages.

Initially we tought about using OpenAI's `TTS` API due to it's impressive performance. However, we soon realized it's performance is really only limited to English. The audio files generated by  OpenAI's `TTS` in romanized Urdu sounded extremely unnatural. Since  OpenAI's `TTS` does not let you set different output languages, it was essentially an English TTS model trying to pronounce words not found in English at all.

We wanted a model specifically trained to produce audio samples on Urdu textual data only, and `facebook/mms-tts-urd-script_arabic` fit all our requirements, as well as being lightweight, which well help to keep our WhatsApp application responsive. We use `facebook/mms-tts-urd-script_arabic` to generate audio samples from the same sentences in the previous section of this notebook.

In [9]:
pipe = pipeline("text-to-audio", model="facebook/mms-tts-urd-script_arabic", device=device)

In [13]:
from IPython.display import Audio

In [17]:
output1 = pipe(result1["text"])
output2 = pipe(result2["text"])
output3 = pipe(result3["text"])

In [31]:
import soundfile as sf

sf.write('urdu_tts_1.wav', output1['audio'].reshape(-1), output1['sampling_rate'])
sf.write('urdu_tts_2.wav', output2['audio'].reshape(-1), output2['sampling_rate'])
sf.write('urdu_tts_3.wav', output3['audio'].reshape(-1), output3['sampling_rate'])

In [32]:
sound1 = IPython.core.display.display(IPython.display.Audio('urdu_tts_1.wav'))
sound2 = IPython.core.display.display(IPython.display.Audio('urdu_tts_2.wav'))
sound3 = IPython.core.display.display(IPython.display.Audio('urdu_tts_3.wav'))

As you can hear, `facebook/mms-tts-urd-script_arabic` produces outputs that sound relatively similar to how a human would speak Urdu. The model has clearly learned various pronounciations subtleties present in native Urdu speakers and outputs audio that, while still distincly 'machine-like', is good enough to be easily understood. We aim to explore other models and APIs for their Urdu STT and TTS capabilities but for now, these extremly lightweight and impressive models; `whisper` and `facebook/mms-tts` for STT and TTS respectively, will be sufficient and welcome additions for increasing access to our application for Urdu-only populations.